In [5]:
import jieba
import re
import jieba.posseg as psg
import jieba.posseg as ps

#读取文件
path = '481676.txt'
with open(path,'r',encoding='utf-8') as f:
    str_doc = f.read()
print('初始文本: \n'+str_doc)

#正则化
#去掉空格字符
str_doc = re.sub('\u3000','',str_doc)#\u3000 是全角的空白符
str_doc = re.sub('\n','',str_doc) #去除换行符
str_doc=re.sub('\s+','',str_doc) #去除空格符


#自定义抽取特征规则
def extract_feature_words(str_doc):
    findWord=' '
    stwlist = get_stop_words()
    user_pos_list = ['nr','ns','nt'] #用户自定义特征词性列表
    for word,pos in ps.cut(str_doc):
        #过滤停用词
        if word not in stwlist and pos in user_pos_list:
            if word+' '+pos+'\n' not in findWord:
                findWord += word+' '+pos+'\n'
    print(findWord)

#创建停用词列表    
def get_stop_words(path=r'stop_words.txt'):
    file = open(path, 'r', encoding='utf-8').read().split('\n')
    return set(file)

#去除停用词
split_list = [[w,t] for w, t in psg.cut(str_doc)
               if w not in get_stop_words(r'stop_words.txt')]


print('-'*90)
print('样本分词结果：\n'+' '.join(['{0}/{1}'.format(w,t) for w, t in split_list]))


#分词去停用词，提取特定词性的实体
extract_feature_words(str_doc)

初始文本: 
电影著作权协会反盗版 联合片方保护权益(组图)
　　新浪娱乐讯 7月30日，中国电影著作权协会，及其有关会员单位，决定就上海宽娱数码科技有限公司(英雄宽频)，杭州世纪联线网络技术有限公司(世纪联线)，上海隐志网络科技有限公司(VeryCD)，以及其他侵权网站的侵权盗版行为，向有关政府主管部门提出行政处罚的请求并向人民法院提起民事诉讼，以切实保护自己的合法利益不受伤害。华谊兄弟首席法律顾问孙健红称，盗版对《唐山大地震》造成的损失不好计算，但可以说无法估量。
　　华谊兄弟就被侵权提起诉讼
　　华谊兄弟首席法律顾问孙健红表示，7月20日就发现了风行网已经有《唐山大地震》的“枪版”在网络传播，随后华谊就做了公正保全工作。由于电影还在热映期，且投资巨大，这些网站的经营者不顾版权人合法权益，为电影在网络上正常的使用传播，造成了巨大的损害。孙健红透露，华谊曾找某网站做了问卷调查，如果没有免费的电影，是否会去电影院看电影，结果42%的人选择会去电影院买票，41%的人选择不去花钱看，其余的无所谓。
　　影著协表示支持会员单位华谊兄弟传媒股份有限公司就影片《唐山大地震》和《谍海风云》开展维权活动。7月22日，《唐山大地震》公映不久就遭遇盗版，与该公司的《谍海风云》不久前遭遇盗版的情况如出一辙。日前，华谊兄弟已取得充分、可靠、有效证据，将向侵权方提起诉讼。
　　《叶问2》片方索赔1000万
　　影著协理事长朱永德表示，将首次以自己的名义起诉北京市若干网吧以及提供侵权的上海“英雄宽频”和杭州“世纪联线”。据了解，在2010年7月21日国家版权局等三部正式启动“剑网行动”之后，上海“英雄宽频”和杭州“世纪联线”仍然提供《枪王之王》、《天下第二》等众多影片给北京的网吧使用。影著协在取得大量确凿证据后，决定分批向管辖法院提起民事诉讼，要求立即停止侵权，赔礼道歉，并赔偿经济损失，这是中国电影著作权协会转变为著作权集体管理组织后提起的第一桩民事诉讼案。
　　此外，影著协和大盛国际传媒(北京)有限公司联手起诉上海隐志网络科技有限公司。据悉，2010年5月4日，电影《叶问2：宗师传奇》上映不久，就发现上海隐志网络公司经营的“电驴”网站首页推荐侵权的清晰版《叶问2》，导致网络出现盗播《叶问2》的局面。影著协与制片方将联合向上海市第一中级人民法院提起诉讼，根据“电驴”的点击率，影院电子售票系统

In [2]:
import re
from datetime import datetime,timedelta
from dateutil.parser import parse
import jieba.posseg as psg

UTIL_CN_NUM = {
    '零': 0, '一': 1, '二': 2, '两': 2, '三': 3, '四': 4,
    '五': 5, '六': 6, '七': 7, '八': 8, '九': 9,
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
    '5': 5, '6': 6, '7': 7, '8': 8, '9': 9
}

UTIL_CN_UNIT = {'十': 10, '百': 100, '千': 1000, '万': 10000}

def cn2dig(src):
    if src == "":
        return None
    m = re.match("\d+", src)
    if m:
        return int(m.group(0))
    rsl = 0
    unit = 1
    for item in src[::-1]:
        if item in UTIL_CN_UNIT.keys():
            unit = UTIL_CN_UNIT[item]
        elif item in UTIL_CN_NUM.keys():
            num = UTIL_CN_NUM[item]
            rsl += num * unit
        else:
            return None
    if rsl < unit:
        rsl += unit
    return rsl

def year2dig(year):
    res = ''
    for item in year:
        if item in UTIL_CN_NUM.keys():
            res = res + str(UTIL_CN_NUM[item])
        else:
            res = res + item
    m = re.match("\d+", res)
    if m:
        if len(m.group(0)) == 2:
            return int(datetime.datetime.today().year/100)*100 + int(m.group(0))
        else:
            return int(m.group(0))
    else:
        return None

def parse_datetime(msg):
    #print('msg:',msg)
    if msg is None or len(msg) == 0:
        return None


    m = re.match(
        r"([0-9零一二两三四五六七八九十]+年)?([0-9一二两三四五六七八九十]+月)?([0-9一二两三四五六七八九十]+[号日])?([上中下午晚早]+)?([0-9零一二两三四五六七八九十百]+[点:\.时])?([0-9零一二三四五六七八九十百]+分?)?([0-9零一二三四五六七八九十百]+秒)?",
             msg)       
    if m.group(0) is not None:
        res = {
            "year": m.group(1),
            "month": m.group(2),
            "day": m.group(3),
            "noon":m.group(4),  # 上中下午晚早
            "hour": m.group(5) if m.group(5) is not None else '00',
            "minute": m.group(6) if m.group(6) is not None else '00',
            "second": m.group(7) if m.group(7) is not None else '00',
        }
        params = {}
        for name in res:
            if res[name] is not None and len(res[name]) != 0:
                tmp = None
                if name == 'year':
                    tmp = year2dig(res[name][:-1])
                else:
                    tmp = cn2dig(res[name][:-1])
                if tmp is not None:
                    params[name] = int(tmp)
        target_date = datetime.today().replace(**params)
        is_pm = m.group(4)
        if is_pm is not None:
            if is_pm == u'下午' or is_pm == u'晚上' or is_pm =='中午':
                hour = target_date.time().hour
                if hour < 12:
                    target_date = target_date.replace(hour=hour + 12)
        return target_date.strftime('%Y-%m-%d %H:%M:%S')
    else:
        return None


# 对提取出的拼接日期串进行进一步的处理，进行有效性判断
def check_time_valid(word):
    m = re.match("\d+$", word)
    if m:
        if len(word) <= 6:
            return None
    word1 = re.sub('[号|日]\d+$', '日', word)
    if word1 != word:
        return check_time_valid(word1)
    else:
        return word1

#时间提取
def time_extract(text):
    time_res = []
    word = ''
    keyDate = {'今天': 0, '明天':1, '后天': 2}
    for k, v in psg.cut(text):
        #print(k,v)
        if k in keyDate:
            if word != '':
                time_res.append(word)  
            # 日期的转换，timedelta提取任意延迟天数的信息
            word = (datetime.today() +timedelta(days=keyDate.get(k, 0))).strftime('%Y年%m月%d日').format(y='年',m='月',d='日') 

        elif word != '':
            if v in ['m', 't']:
                word = word + k
            else:
                time_res.append(word)
                word = ''
        elif v in ['m', 't']:  # m:数字 t:时间
            word = k            

    if word != '':
        time_res.append(word)
    # filter() 函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表
    result = list(filter(lambda x: x is not None, [check_time_valid(w) for w in time_res]))
    final_res = [parse_datetime(w) for w in result]
    return [x for x in final_res if x is not None]

#测试
text1 = '2012年7月22日5点'
print(text1, time_extract(text1), sep=',')

text2 = '明天下午四点'
print(text2, time_extract(text2), sep=',')

text3 = '预定28号下午四点的房间'
print(text3, time_extract(text3), sep=',')


text4 = '我要预定今天到后天的房间'
print(text4, time_extract(text4), sep=',')

text5 = '那我们可以预定明天下午4点到后天晚上8点的宾馆'
print(text5, time_extract(text5), sep=',')


2012年7月22日5点,['2012-07-22 05:00:00']
明天下午四点,['2022-11-22 16:00:00']
预定28号下午四点的房间,['2022-11-28 16:00:00']
我要预定今天到后天的房间,['2022-11-21 00:00:00', '2022-11-23 00:00:00']
那我们可以预定明天下午4点到后天晚上8点的宾馆,['2022-11-22 16:00:00', '2022-11-23 20:00:00']


In [268]:
print("筛除文本中的日期",time_extract(str_doc), sep=',')

筛除文本中的日期 ['2022-10-01 02:20:00', '2022-07-30 00:00:00', '2022-07-20 00:00:00', '2022-10-01 00:00:00', '2022-07-22 00:00:00', '2022-10-01 02:20:00', '2010-07-21 00:00:00', '2010-05-04 00:00:00', '2022-10-01 00:00:00']
